In [1]:
from dataclasses import dataclass, field
from typing import List, Optional, Union
from collections import defaultdict
import json
import os
import string
from string import ascii_letters
from textwrap import dedent

import numpy as np
import pandas as pd

In [47]:
data = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k""".strip()

In [86]:
def parse_process_list(input_stream) -> List[List[str]]:
    """Parse input data into commands."""
    current_line_idx = 0
    current_line = None
    parsed_commands_list = []
    cwd = ""
    process_list = list(input_stream.splitlines())
    while current_line_idx < len(process_list) - 1:
        current_line = process_list[current_line_idx]
        
        if current_line[0] == "$":
            # Line is a command.

            if current_line[2:4] == "cd":
                # Change directory...
                split_command = current_line.split(" ")
                cwd = os.path.abspath(os.path.join(cwd, split_command[2]))
                parsed_commands_list.append(split_command[1:3])

                current_line_idx += 1

            elif current_line[2:4] == "ls":
                # List directory... 
                ls_cmd = ["ls", cwd]
                
                while (
                    current_line_idx < len(process_list) - 1
                    and process_list[current_line_idx + 1][0] != "$"         
                ):
                    current_line_idx += 1
                    current_line = process_list[current_line_idx]
                    ls_cmd.append(current_line)
                
                parsed_commands_list.append(ls_cmd)
                current_line_idx += 1
            else:
                current_line_idx += 1
        else:
            raise ValueError("Should not be processing a non-command.")

    return parsed_commands_list

In [101]:
@dataclass
class File:
    """Represent a File which will have a name, extension, and size."""
    name: str
    extension: str
    size: int


@dataclass
class Directory:
    """Represent a directory which may contain other ``Directory``s and ``File``s."""
    name: str
    parent: Optional['Directory']
    children: Optional[List[Union['Directory', File]]]

@dataclass
class FileSystem:
    """Represents the file system containing ``Directory``s and ``File``s."""

    dirs: List[Directory]

    def add_child_directory(self, target_dir: Directory):
        if target_dir.name not in [d.name for d in self.dirs if isinstance(d, Directory)]:
            self.dirs.append(target_dir)
            target_dir.parent.children.append(target_dir)
    
    def add_files_to_directory(self, target_dir: Directory, file: File):
        if file.name not in [f for f in target_dir.children if isinstance(f, File)]:
            target_dir.children.append(file)

    def get_dir_by_name(self, dirname:str) -> Directory:
        return [d for d in self.dirs if d.name == dirname][0]

    def calculate_dir_size(self, directory_name:str) -> int:
        size = 0
        directory = self.get_dir_by_name(directory_name)
        for item in directory.children:
            if isinstance(item, File):
                size += item.size
            else:
                size += self.calculate_dir_size(item.name)
        return size


In [102]:
parsed_cmds = parse_process_list(data)

fs = FileSystem(dirs=[Directory(name="/", parent=None, children=[])])
cwd = '/'
for cmd in parsed_cmds:
    if cmd[0] == "cd":
        new_dir_loc = os.path.abspath(os.path.join(cwd, cmd[1]))
        new_dir = Directory(name=new_dir_loc, parent=fs.get_dir_by_name(cwd), children=[])
        fs.add_child_directory(new_dir)
        cwd = new_dir_loc
    if cmd[0] == "ls":
        cwd = cmd[1]
        for item in cmd[2:]:
            # if item[:4] == "dir ":
            #     new_dir = Directory(name=cwd, parent=os.path.abspath(os.path.join(cwd, "..")), children=[])
            #     fs.add_child_directory(item[4:], cwd)
            if item[:4] != "dir ":
                size, name = item.split(" ")
                size = int(size)
                if "." in name:
                    name, ext = name.split(".")
                else:
                    ext = ''
                current_dir = fs.get_dir_by_name(cwd)
                fs.add_files_to_directory(current_dir, File(name=name, extension=ext, size=size))


In [110]:
total = 0
for d in [d for d in fs.dirs if isinstance(d, Directory)]:
    if val:= fs.calculate_dir_size(d.name) >= 6728267:
        print(fs.calculate_dir_size(d.name))
        total += fs.calculate_dir_size(d.name)

print(total)


46728267
32283214
7951177
13880430
7490863
10862928
119196879


In [104]:
data = """$ cd /
$ ls
dir cvt
4967 hcqbmwc.gts
5512 hsbhwb.clj
dir hvfvt
dir phwgv
277125 pwgswq.fld
42131 qdzr.btl
dir svw
144372 vmbnlzgb.wbd
dir zft
$ cd cvt
$ ls
dir bbgsthsd
146042 bcqrmp.czf
dir chhdjtlw
dir cpcfcc
dir dch
dir djb
dir djfww
dir drdf
dir fgbqtjlj
dir hjsmmj
243293 hvfvt.qtb
245795 lrpb
dir msptbrl
181756 qlqqmndd.zcb
18658 rtfzt.tjp
dir slpc
$ cd bbgsthsd
$ ls
236957 djfww.fcb
112286 hcqbmwc.gts
106102 qggjrzts
$ cd ..
$ cd chhdjtlw
$ ls
226927 cpcfcc
309815 djfww
117933 hcqbmwc.gts
dir mbdrgfzs
dir pbmcnpzf
131558 pwgswq.fld
298691 qlqqmndd.zcb
$ cd mbdrgfzs
$ ls
164331 hvfvt.dvq
$ cd ..
$ cd pbmcnpzf
$ ls
102120 bjgg.cqd
$ cd ..
$ cd ..
$ cd cpcfcc
$ ls
15756 ddc
dir dqc
dir glm
dir jbszm
200345 pwgswq.fld
145508 qlqqmndd.zcb
dir vcptbw
dir zrtm
$ cd dqc
$ ls
dir cpcfcc
92063 mzp
dir qmhnvmh
dir snqqcjlw
8423 zdjwr.blc
$ cd cpcfcc
$ ls
dir hvfvt
$ cd hvfvt
$ ls
289372 pwgswq.fld
307397 qlqqmndd.zcb
$ cd ..
$ cd ..
$ cd qmhnvmh
$ ls
274778 jczjwdl.smd
194322 ngrrfm
$ cd ..
$ cd snqqcjlw
$ ls
182851 nprvrgd.tbb
$ cd ..
$ cd ..
$ cd glm
$ ls
188702 hpq.hgm
$ cd ..
$ cd jbszm
$ ls
187645 pwgswq.fld
$ cd ..
$ cd vcptbw
$ ls
dir mgqd
$ cd mgqd
$ ls
157695 rhldvntj.jzm
268696 rjnngctw
$ cd ..
$ cd ..
$ cd zrtm
$ ls
218820 bcqrmp.czf
172319 czpsl.dnf
180114 hcqbmwc.gts
165216 hvfvt.nnw
dir zft
$ cd zft
$ ls
161245 dtqjg.czv
249888 hpq.hgm
192037 zft
$ cd ..
$ cd ..
$ cd ..
$ cd dch
$ ls
dir djfww
dir fzqgwwnf
dir hqvmq
dir mmcrzqc
42136 qlqqmndd.zcb
dir vdtlh
dir wfltm
271200 zrtm
$ cd djfww
$ ls
37427 dct
204401 hcqbmwc.gts
283140 jmq
dir lbrhbc
154148 pncb.bmp
53366 vzt.cnn
$ cd lbrhbc
$ ls
dir djfww
153648 djfww.pps
308911 mvtjzp.btg
219150 ncmc
253604 wqft.ggb
293759 zft
$ cd djfww
$ ls
24794 hcqbmwc.gts
$ cd ..
$ cd ..
$ cd ..
$ cd fzqgwwnf
$ ls
233720 scdd.pwr
$ cd ..
$ cd hqvmq
$ ls
dir cdld
dir cpcfcc
dir djfww
85564 hpq.hgm
dir qnc
81509 wtnl.crs
$ cd cdld
$ ls
293421 llpltsgn.tqb
$ cd ..
$ cd cpcfcc
$ ls
230813 cglljgv
178203 qlqqmndd.zcb
186410 zrtm.tbt
$ cd ..
$ cd djfww
$ ls
201830 zrtm
$ cd ..
$ cd qnc
$ ls
305356 bdf.jrh
dir zrtm
$ cd zrtm
$ ls
dir bdhdrr
$ cd bdhdrr
$ ls
dir nffdrnb
$ cd nffdrnb
$ ls
301601 nbrjz
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd mmcrzqc
$ ls
dir cpcfcc
264271 hpq.hgm
208744 tzsjrf.zdc
274096 zft
$ cd cpcfcc
$ ls
226054 cpcfcc
$ cd ..
$ cd ..
$ cd vdtlh
$ ls
84229 bzlg.crn
58668 cpcfcc.zqw
141409 hpq.hgm
dir mdgst
37090 pstsh.qwv
40357 qlqqmndd.zcb
dir smr
dir sqbgcsgh
dir trp
$ cd mdgst
$ ls
151033 fsmvbpsl.nqr
$ cd ..
$ cd smr
$ ls
140764 cpcfcc.dpg
$ cd ..
$ cd sqbgcsgh
$ ls
dir djfww
263783 dvm
105219 hpq.hgm
29647 hwcgv.gvg
162537 lwjhgh.wmt
235511 nlw
dir zwddrdfz
$ cd djfww
$ ls
260084 vcgpvpd
$ cd ..
$ cd zwddrdfz
$ ls
225632 bhrzzw
269756 cqq.vrz
$ cd ..
$ cd ..
$ cd trp
$ ls
233192 vqwngcs
$ cd ..
$ cd ..
$ cd wfltm
$ ls
140990 djfww
$ cd ..
$ cd ..
$ cd djb
$ ls
213191 cjwrs.prd
dir dhvpqt
dir gtgftm
dir hvfvt
208325 jwhp
301446 qlqqmndd.zcb
dir qnlbgvf
dir tps
dir zft
dir zrtm
dir zwnlm
$ cd dhvpqt
$ ls
dir sbcj
$ cd sbcj
$ ls
20611 ggt
$ cd ..
$ cd ..
$ cd gtgftm
$ ls
179526 qcqbtmvq
$ cd ..
$ cd hvfvt
$ ls
dir djstcnrt
dir hggpvn
dir hvfvt
63889 hvfvt.tjm
265836 qlqqmndd.zcb
105663 zft
223582 zft.llg
26037 zzwwg.lqh
$ cd djstcnrt
$ ls
201498 hcqbmwc.gts
139156 hpq.hgm
277342 pshgz.rtp
dir rjb
257294 rqrb.lhh
dir tqnbfnt
313308 zft.dvc
$ cd rjb
$ ls
268438 hpq.hgm
dir hvfvt
dir hvwpb
161024 mzbcgqc.nvw
137875 pwgswq.fld
260008 pzzcww
283261 wnvlrg.tmn
$ cd hvfvt
$ ls
dir dvwqqt
163397 hcqbmwc.gts
265154 pwgswq.fld
dir trsh
dir vnf
$ cd dvwqqt
$ ls
315187 hcqbmwc.gts
$ cd ..
$ cd trsh
$ ls
dir ssrdpg
$ cd ssrdpg
$ ls
dir vjdc
$ cd vjdc
$ ls
dir srvdrhdq
$ cd srvdrhdq
$ ls
74145 djfww
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd vnf
$ ls
dir djfww
$ cd djfww
$ ls
184018 pwgswq.fld
$ cd ..
$ cd ..
$ cd ..
$ cd hvwpb
$ ls
dir jwpjrcvs
$ cd jwpjrcvs
$ ls
dir cpcfcc
$ cd cpcfcc
$ ls
52191 sdlv
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd tqnbfnt
$ ls
dir djfww
206341 fmpb.lnp
dir fpsjg
4589 hcqbmwc.gts
dir hvfvt
8796 hvfvt.hdz
238364 tgcdpjc.fjm
dir zrtm
40577 zrtm.fsr
$ cd djfww
$ ls
217182 djfww.rbf
$ cd ..
$ cd fpsjg
$ ls
52456 djfww
$ cd ..
$ cd hvfvt
$ ls
dir rqhzvb
275026 szwgzm.qjd
dir zbmtstj
$ cd rqhzvb
$ ls
8395 gcmzsd.jzf
$ cd ..
$ cd zbmtstj
$ ls
39691 qlqqmndd.zcb
dir qqcdss
180676 rwl.zwr
$ cd qqcdss
$ ls
106297 zrtm.nsl
$ cd ..
$ cd ..
$ cd ..
$ cd zrtm
$ ls
153478 ldhvt
$ cd ..
$ cd ..
$ cd ..
$ cd hggpvn
$ ls
19109 cpcfcc.fbh
249168 fnqjc
207420 qgzsh.pfs
135472 qlqqmndd.zcb
$ cd ..
$ cd hvfvt
$ ls
dir djfww
213659 hcqbmwc.gts
dir jzdwq
204021 nvbgf
185328 qlqqmndd.zcb
dir zrtm
$ cd djfww
$ ls
180713 rwmm.gjt
$ cd ..
$ cd jzdwq
$ ls
57424 djfww.qtm
104960 slq.bfp
174906 svc
$ cd ..
$ cd zrtm
$ ls
dir mrtvhht
$ cd mrtvhht
$ ls
188512 djfww.fvm
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd qnlbgvf
$ ls
dir fjrhtssv
46767 jhbnml.lpq
$ cd fjrhtssv
$ ls
198664 fgcc
245142 tsc.rsd
$ cd ..
$ cd ..
$ cd tps
$ ls
dir fmlwghjv
146731 gwch.ttr
dir jjzmdd
dir rtltrng
191609 wff.mqt
$ cd fmlwghjv
$ ls
189227 zrtm.vgg
$ cd ..
$ cd jjzmdd
$ ls
dir hvfvt
178057 mpdfrrm
49493 pwgswq.fld
220129 sqgfb.flm
231012 zrmsnmr
$ cd hvfvt
$ ls
dir lbndbhc
12602 mnclbss
746 ppw
$ cd lbndbhc
$ ls
49601 cdqvs.gwc
158670 wvjpchjg
$ cd ..
$ cd ..
$ cd ..
$ cd rtltrng
$ ls
dir cpgh
115613 hvfvt
59441 zfs
$ cd cpgh
$ ls
dir djfww
$ cd djfww
$ ls
dir tgt
$ cd tgt
$ ls
dir vqwvqss
$ cd vqwvqss
$ ls
dir bsdrhz
$ cd bsdrhz
$ ls
dir pld
$ cd pld
$ ls
8543 mfs.gsb
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd zft
$ ls
dir fhjgg
$ cd fhjgg
$ ls
114494 pwgswq.fld
$ cd ..
$ cd ..
$ cd zrtm
$ ls
201142 tlhr.rtd
298232 wlcztszv
$ cd ..
$ cd zwnlm
$ ls
240023 hcqbmwc.gts
159126 hpq.hgm
22101 hvfvt.bfw
dir ltzl
dir mtfr
dir sbw
dir sqscdjlj
dir ssf
$ cd ltzl
$ ls
260692 hpq.hgm
204540 hvfvt
$ cd ..
$ cd mtfr
$ ls
15391 pwgswq.fld
$ cd ..
$ cd sbw
$ ls
293226 hcqbmwc.gts
267478 htllcq
176349 nlbhcf.vpn
286577 zft.gds
$ cd ..
$ cd sqscdjlj
$ ls
128776 gnh
dir qbdpvcbt
284505 qlsn.vdq
$ cd qbdpvcbt
$ ls
280709 crv.hgm
$ cd ..
$ cd ..
$ cd ssf
$ ls
13328 djfww.pjn
19971 hvfvt
dir zft
$ cd zft
$ ls
97761 dnmnz
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd djfww
$ ls
109590 bwfnnf.wpd
297297 cmht.ljg
280726 hvfvt
$ cd ..
$ cd drdf
$ ls
56764 pwgswq.fld
300547 wqqm
210698 zrtm.jvq
$ cd ..
$ cd fgbqtjlj
$ ls
22217 vrmgsz.ctg
$ cd ..
$ cd hjsmmj
$ ls
232911 bcqrmp.czf
164656 hcqbmwc.gts
dir jql
dir ntd
dir prgm
dir psfvhh
dir zft
$ cd jql
$ ls
212767 zhnhn.lzm
dir zrtm
$ cd zrtm
$ ls
35950 bcqrmp.czf
$ cd ..
$ cd ..
$ cd ntd
$ ls
dir vml
$ cd vml
$ ls
60833 bcqrmp.czf
$ cd ..
$ cd ..
$ cd prgm
$ ls
316166 pwgswq.fld
$ cd ..
$ cd psfvhh
$ ls
152195 pwgswq.fld
92448 qlqqmndd.zcb
$ cd ..
$ cd zft
$ ls
95808 hpq.hgm
$ cd ..
$ cd ..
$ cd msptbrl
$ ls
256069 hpq.hgm
$ cd ..
$ cd slpc
$ ls
dir djfww
285380 djfww.gcm
169898 hcqbmwc.gts
40914 mnpnhpz
188062 pwgswq.fld
dir zrtm
$ cd djfww
$ ls
94935 pwgswq.fld
70655 qlqqmndd.zcb
48528 zft.qvh
$ cd ..
$ cd zrtm
$ ls
160137 hcqbmwc.gts
$ cd ..
$ cd ..
$ cd ..
$ cd hvfvt
$ ls
dir cslthslv
129407 prwq
dir vrzfbtt
$ cd cslthslv
$ ls
dir cpcfcc
158361 hcqbmwc.gts
$ cd cpcfcc
$ ls
dir zpgnczq
$ cd zpgnczq
$ ls
158414 zft
$ cd ..
$ cd ..
$ cd ..
$ cd vrzfbtt
$ ls
185263 nqvvl
$ cd ..
$ cd ..
$ cd phwgv
$ ls
253690 bzpcj.fcj
dir wtbr
168025 zdsh.tnq
157840 zsdfqb.zbd
$ cd wtbr
$ ls
dir lvgdb
$ cd lvgdb
$ ls
177751 cpcfcc.dzr
$ cd ..
$ cd ..
$ cd ..
$ cd svw
$ ls
dir cpcfcc
189430 hpq.hgm
dir ntzbj
13384 qlqqmndd.zcb
10333 wdvqhwgl.trq
161318 zwzp.jsn
$ cd cpcfcc
$ ls
298999 hcqbmwc.gts
290883 hpq.hgm
dir qlnntp
dir vpshs
5547 wlrmg.bpc
dir zft
$ cd qlnntp
$ ls
168562 nqds.tqn
$ cd ..
$ cd vpshs
$ ls
246400 pwgswq.fld
$ cd ..
$ cd zft
$ ls
235634 hvfvt.hst
$ cd ..
$ cd ..
$ cd ntzbj
$ ls
98777 qlqqmndd.zcb
$ cd ..
$ cd ..
$ cd zft
$ ls
dir dhnzm
299292 hvfvt.pvb
dir ltfb
dir mrvsjgsq
dir ntr
dir swmvhgd
dir zrtm
$ cd dhnzm
$ ls
267041 bbsj
$ cd ..
$ cd ltfb
$ ls
242136 bcqrmp.czf
314862 dgdz
203160 hcqbmwc.gts
80422 hgvz
dir qftjfqfv
180815 sstl.wzg
dir wmhm
dir zft
dir zszp
$ cd qftjfqfv
$ ls
dir bmmw
$ cd bmmw
$ ls
dir hvfvt
318039 hvfvt.whc
17057 pwgswq.fld
$ cd hvfvt
$ ls
170348 brplf.wgv
156783 hcqbmwc.gts
$ cd ..
$ cd ..
$ cd ..
$ cd wmhm
$ ls
178474 bcqrmp.czf
dir cmsgv
dir dcfvrtj
225393 htjflrs.tlm
dir hvfvt
dir nlrs
273173 nsmdd.glc
151259 pwgswq.fld
dir ztqvwjr
$ cd cmsgv
$ ls
dir flgjbjg
$ cd flgjbjg
$ ls
13971 cpcfcc
20601 qlqqmndd.zcb
$ cd ..
$ cd ..
$ cd dcfvrtj
$ ls
141992 djfww.djn
118149 hpq.hgm
dir qrmqslq
$ cd qrmqslq
$ ls
105940 zrtm.vql
$ cd ..
$ cd ..
$ cd hvfvt
$ ls
181962 hpq.hgm
$ cd ..
$ cd nlrs
$ ls
82705 vnfwjf.hhl
$ cd ..
$ cd ztqvwjr
$ ls
98257 pwgswq.fld
83864 scv.pbp
$ cd ..
$ cd ..
$ cd zft
$ ls
183072 rsbfsl.zsj
$ cd ..
$ cd zszp
$ ls
dir hvfvt
$ cd hvfvt
$ ls
dir djfww
dir dqdtgjtg
dir hvfvt
$ cd djfww
$ ls
117037 rnmbsq.zph
259243 zrtm.znf
$ cd ..
$ cd dqdtgjtg
$ ls
dir lrbsvng
59301 zft
$ cd lrbsvng
$ ls
46301 bcqrmp.czf
$ cd ..
$ cd ..
$ cd hvfvt
$ ls
223277 qlqqmndd.zcb
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd mrvsjgsq
$ ls
160189 cpcfcc
dir lzqf
232626 pwgswq.fld
dir rpmfcg
$ cd lzqf
$ ls
135552 bcqrmp.czf
48083 qlqqmndd.zcb
dir rbg
229086 rdswb.cjz
204486 spbwfrwf.cmj
$ cd rbg
$ ls
dir sjdwsbqq
$ cd sjdwsbqq
$ ls
270621 rhv.shw
$ cd ..
$ cd ..
$ cd ..
$ cd rpmfcg
$ ls
227216 bld.jtn
dir zft
$ cd zft
$ ls
43056 vzc
$ cd ..
$ cd ..
$ cd ..
$ cd ntr
$ ls
dir cmtnqb
dir djfww
195371 wffvmql.lrv
$ cd cmtnqb
$ ls
135380 pbng.vhq
$ cd ..
$ cd djfww
$ ls
dir fmvrrrq
dir mbhrbss
dir zrtm
$ cd fmvrrrq
$ ls
dir cpcfcc
79110 djfww
300581 ftcdj.wcc
$ cd cpcfcc
$ ls
234211 zsgsm
$ cd ..
$ cd ..
$ cd mbhrbss
$ ls
149184 djfww.gwh
185815 rhgw.rmj
$ cd ..
$ cd zrtm
$ ls
dir zft
$ cd zft
$ ls
310043 bnwvt.fbg
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd swmvhgd
$ ls
77547 djfww.jwr
318547 drggg
300962 pwgswq.fld
$ cd ..
$ cd zrtm
$ ls
229319 bflnffjj
dir djfww
142526 hvfvt
232722 jprrf
94024 ndmf.hdr
dir vhdhn
$ cd djfww
$ ls
30770 bcqrmp.czf
$ cd ..
$ cd vhdhn
$ ls
dir djfww
48261 djfww.frg
dir gqc
316163 qlqqmndd.zcb
$ cd djfww
$ ls
104095 cvjlthn.cfl
223454 hgqwzh
24715 pwgswq.fld
283499 qlqqmndd.zcb
dir zft
224574 zlnwn
$ cd zft
$ ls
100941 pwgswq.fld
$ cd ..
$ cd ..
$ cd gqc
$ ls
156273 wpgwrdl""" 


In [105]:
fs.calculate_dir_size("/")

46728267

In [107]:
70000000 - 30000000 - 46728267 

-6728267

In [ ]:
6728267